<a href="https://colab.research.google.com/github/ahmeda335/Stories-Using-LLMs/blob/main/ChildrenStoriesUisngLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating English And Arabic Stories Using LLMs 😎.

⚡ Installing required packages.

In [1]:
!pip install -q -U google-generativeai
!pip install -q python-docx
!pip install -q googletrans==4.0.0-rc1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.0 MB/s eta 0:00:00


⚡ Importing required libraries.

In [2]:
import google.generativeai as genai
import os
import json
import random
import time

import requests
import io
from PIL import Image
from IPython.display import display

from huggingface_hub import login, HfApi
from tenacity import retry, wait_random_exponential, stop_after_attempt

***Your Gemini-API-Key***<br>
⚡ In the case of you are using colab, you can set your API-KEY in this way in the cell. If You are using any other development environment or cURL in the terminal it is preferred to put the API-KEY in an environment variable as shown in the next cell. <br>
⚡ As I am using colab I will use the first method.

You can get your Gemini API Key from here --> https://aistudio.google.com/app/apikey

In [3]:
from google.colab import userdata
API_KEY = userdata.get('GEMINI-API-KEY')

⚡ If you don't use google colab.

In [ ]:
# from dotenv import load_dotenv
# load_dotenv()

# !export GEMINI-API-KEY = <YOUR_API_KEY>   # Replace '<YOUR_API_KEY>' with you API-KEY. If you are using the terminal don't use '!' before 'export'.
# API_KEY = os.getenv('GEMINI-API-KEY')

***Your Hugging Face token***<br>
⚡ You can get your Hugging Face token from here --> https://huggingface.co/settings/tokens <br>
⚡ You may need it in case of gated models.

In [4]:
# Getting my HF-TOKEN from secret section
token= userdata.get('HF-TOKEN')
login(token)

# Create an instance of the HfApi class
api = HfApi()

# Get user information
user_info = api.whoami()

# Print user information to verify
print("\nYour account:", user_info['name'])

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful

Your account: ahmeda335


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


⚡ Inferencing text-to-image model.

In [5]:
API_URL = "https://api-inference.huggingface.co/models/black-forest-labs/FLUX.1-schnell"
headers = {"Authorization": f"Bearer {token}"}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.content

⚡ Creating a Gemini model with my token.

In [6]:
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash')

⚡ Places such that stories happen in.

In [7]:
# These are 100 places for the stories.
places = [
    "Enchanted Forest",
    "Magical Castle",
    "Hidden Waterfall",
    "Sunny Beach",
    "Mysterious Cave",
    "Ancient Pyramid",
    "Space Station",
    "Desert Oasis",
    "Snowy Mountain",
    "Underwater City",
    "Secret Garden",
    "Hot Air Balloon",
    "Haunted Mansion",
    "Rainforest Canopy",
    "Floating Island",
    "Mystical Temple",
    "Forgotten Village",
    "Giant's Playground",
    "Pirate Ship",
    "Candy Land",
    "Dinosaur Valley",
    "Crystal Cave",
    "Lunar Base",
    "Toy Workshop",
    "Butterfly Meadow",
    "Fairy Village",
    "Coral Reef",
    "Cloud Kingdom",
    "Robot Factory",
    "Ice Palace",
    "Wishing Well",
    "Dragon's Lair",
    "Sunken Ship",
    "Golden Desert",
    "Mountainous Cliff",
    "Echoing Canyon",
    "Firefly Forest",
    "Rainbow Bridge",
    "Mystery Island",
    "Secret Library",
    "Clock Tower",
    "Bamboo Grove",
    "Bubblegum Swamp",
    "Meteor Shower",
    "Crystal Lake",
    "Spooky Swamp",
    "Treehouse Village",
    "Windmill Farm",
    "Time Traveler's Den",
    "Magician's Workshop",
    "Silver Mine",
    "Floating Cloud",
    "Northern Lights",
    "Jungle Treehouse",
    "Spaceport",
    "River Rapids",
    "Magic Carpet Ride",
    "Lighthouse",
    "Mossy Cave",
    "Labyrinth",
    "Gemstone Quarry",
    "Volcano Crater",
    "Wildflower Field",
    "Candlelit Cave",
    "Carousel Park",
    "Penguin Island",
    "Ancient Ruins",
    "Moonlit Beach",
    "Tropical Rainforest",
    "Wizard's Tower",
    "Polar Ice Cap",
    "Fantasy Forest",
    "Aurora Borealis",
    "Dragon's Nest",
    "Sandy Dunes",
    "Glowing Grotto",
    "Crystal Kingdom",
    "Sunflower Field",
    "Galactic Voyage",
    "Frozen Tundra",
    "Secret Passage",
    "Thunderstorm Plains",
    "Whispering Woods",
    "Mermaid Lagoon",
    "Iceberg",
    "Pumpkin Patch",
    "Fireworks Festival",
    "Magical Market",
    "Peppermint Lane",
    "Whale Watching",
    "Geyser Field",
    "Charming Cottage",
    "Eagle's Nest",
    "Starry Sky",
    "Marshmallow Meadow",
    "Rocky Mountain Pass",
    "Shooting Star Ridge",
    "Cactus Canyon",
    "Rolling Hills",
    "Butterfly Garden",
    "Treasure Cove"
]


In [8]:

# Create a json format to store stories in a json file.
english_stories = []
arabic_stories = []

# Create a folder for images if it doesn't exist.
images_folder_path = '/content/images'
if not os.path.exists(images_folder_path):
    os.makedirs(images_folder_path)

In [9]:
places_copy = places.copy()

# Function to select a random place and remove it from the list
def get_random_place(places_list):
    if places_list:  # Check if the list is not empty
        place = random.choice(places_list)  # Select a random place
        places_list.remove(place)  # Remove the selected place from the list
        return place
    else:
        return "No more places left!"

⚡ The main function.

In [10]:
stories_num = 20
counter = 1
while counter <= stories_num:
  # Getting a random place.
  place = get_random_place(places_copy)
  if place == "No more places left!":
    break

  try:
    # Getting the story.
    story = model.generate_content(f"Write a story from 4 big paragraphs about a child boy called Ramy and his small sister called Lamia playing in the {place}. Using simple words for children.")
    # print(story.text)

    # Getting the summarization of the story.
    summary = model.generate_content(f"Summarize the following story: {story.text}")
    # print(summary.text)

    # Getting the title of the story
    title = model.generate_content(f"What is the title of the following story: '{story.text}'. Give only one title. The title could be: ")
    # print(title.text)

    # Getting the translation of the title.
    arabic_title = model.generate_content(f"Translate the following title to Arabic: '{title.text}'. The arabic title of {title.text} is: ")
    # print(title.text)

    # Getting translation of the story.
    translation = model.generate_content(f"Translate the following story to Arabic: '{story.text}'. Arabic translation is: ")
    # print(translation.text)

    # Sleeping for a minute each two tournamets because of limited tries in Gemini (15 inference/min).
    if counter % 2 == 0:
      time.sleep(60)

    # Getting an image for the story.
    try:
      image_bytes = query({
        "inputs": f"{summary.text}. Such that the boy with red shirt and the girl with blue shirt and the characters are cartoonic.",
      })
      # You can access the image with PIL.Image for example
      @retry(wait=wait_random_exponential(multiplier=1, max=30), stop=stop_after_attempt(5)) # This line to retry the function each random time between 1 and 40 seconds with maximum 3 attempts. To increase the Robustness of the code.
      def store_image(image_bytes):
        image = Image.open(io.BytesIO(image_bytes))
        # Save the image to the folder
        image_save_path = os.path.join(images_folder_path, f'{counter}.jpg')
        image.save(image_save_path)

      store_image(image_bytes)
      counter += 1

    except Exception as e:
      print("exception 1:", e)
      continue

    # Storing the story in the json file.
    story = {
          "number": f"{counter - 1}",
          "story": f"{story.text}",
          "summary": f"{summary.text}",
          "title": f"{title.text}",
          "arabic_title": f"{arabic_title.text}",
          "translation": f"{translation.text}",
          "story_place": f"{place}",
          }
    english_stories.append(story)

    # Save to JSON file.
    with open('stories.json', 'w') as f:
        json.dump(english_stories, f, indent=4)

    print(f"Story {counter - 1} saved successfully.")

  except Exception as e:
    print("exception 2:", e)
    continue

Story 1 saved successfully.
Story 2 saved successfully.
Story 3 saved successfully.
Story 4 saved successfully.
Story 5 saved successfully.
Story 6 saved successfully.
Story 7 saved successfully.
Story 8 saved successfully.
exception 2: Invalid operation: The `response.text` quick accessor requires the response to contain a valid `Part`, but none were returned. Please check the `candidate.safety_ratings` to determine if the response was blocked.
Story 10 saved successfully.
Story 11 saved successfully.
exception 1: Invalid operation: The `response.text` quick accessor requires the response to contain a valid `Part`, but none were returned. Please check the `candidate.safety_ratings` to determine if the response was blocked.
Story 12 saved successfully.
Story 13 saved successfully.
Story 14 saved successfully.
Story 15 saved successfully.
exception 1: Invalid operation: The `response.text` quick accessor requires the response to contain a valid `Part`, but none were returned. Please che

### Putting both Arabic and English stories in word files.

In [11]:
from docx import Document
from docx.shared import Inches
from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH, WD_PARAGRAPH_ALIGNMENT
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from docx.enum.section import WD_SECTION_START, WD_ORIENTATION


In [12]:
doc = Document()

with open('stories.json', 'r') as f:
    english_stories = json.load(f)

for i in range(len(english_stories)):
  heading1 = doc.add_heading(f"{i + 1}", level=1)
  # Center the heading
  heading1.alignment = WD_ALIGN_PARAGRAPH.CENTER
  # Increase the font size
  heading1.style.font.size = Pt(24)

  heading2 = doc.add_heading(f"The { english_stories[i]['story_place']}", level=1)
  # Center the heading
  heading2.alignment = WD_ALIGN_PARAGRAPH.CENTER
  # Increase the font size
  heading2.style.font.size = Pt(24)

  # Insert a line break directly after the heading
  run = heading2.add_run()
  run.add_break()

  stories_num = english_stories[i]['number']
  picture = doc.add_picture(f'/content/images/{stories_num}.jpg', width=Inches(3))
  # Center the picture.
  picture.alignment = WD_ALIGN_PARAGRAPH.CENTER

  # Center the paragraph that contains the picture
  last_paragraph = doc.paragraphs[-1]  # Get the last paragraph (where the picture was added)
  last_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

  paragraph = doc.add_paragraph(english_stories[i]['story'])
  # paragraph.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY  # Justify alignment

   # Set the line spacing to 1.5
  paragraph_format = paragraph.paragraph_format
  paragraph_format.line_spacing = Pt(18)  # 1.5 lines

  # # Center the paragraph horizontally
  paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER

  # # Modify the section properties for vertical alignment
  # section = doc.sections[-1]  # Get the last section
  # section.start_type = WD_SECTION_START.CONTINUOUS
  # section.orientation = WD_ORIENTATION.PORTRAIT
  # section.page_height = section.page_height
  # section.page_width = section.page_width

  # # Set vertical alignment to center
  # section.vertical_alignment = WD_ALIGN_PARAGRAPH.CENTER



  # Set space before and after the paragraph
  paragraph_format.space_before = Pt(12)  # Space before the paragraph
  paragraph_format.space_after = Pt(12)   # Space after the paragraph

  # Set the font size and font type
  for run in paragraph.runs:
      run.font.size = Pt(12)           # Set the font size to 12 points
      run.font.name = 'Cavolini'          # Set the font type to Arial
  # if i != len(english_stories) - 1:
  #   doc.add_page_break()



In [13]:
# # Access the section where you want to add the border
# section = doc.sections[0]

# # Create the page border elements with dark blue and dashed lines
# pgBorders = OxmlElement('w:pgBorders')
# for border_name in ['top', 'left', 'bottom', 'right']:
#     border = OxmlElement(f'w:{border_name}')
#     border.set(qn('w:val'), 'dashed')   # Dashed line border
#     border.set(qn('w:sz'), '12')        # Size of the border (adjust as needed)
#     border.set(qn('w:space'), '24')     # Space between border and text
#     border.set(qn('w:color'), '00008B') # Dark blue color (hex code: #00008B)
#     pgBorders.append(border)

# # Append the page borders to the section properties
# section_properties = section._sectPr
# section_properties.append(pgBorders)

In [14]:
# Add a footer with page numbering
section = doc.sections[0]
footer = section.footer
paragraph = footer.paragraphs[0]

# Create the field code for page numbering
field_code = 'PAGE  \\* MERGEFORMAT'
run = paragraph.add_run()
fldChar1 = OxmlElement('w:fldChar')
fldChar1.set(qn('w:fldCharType'), 'begin')

instrText = OxmlElement('w:instrText')
instrText.set(qn('xml:space'), 'preserve')
instrText.text = field_code

fldChar2 = OxmlElement('w:fldChar')
fldChar2.set(qn('w:fldCharType'), 'separate')

fldChar3 = OxmlElement('w:t')
fldChar3.text = "1"

fldChar4 = OxmlElement('w:fldChar')
fldChar4.set(qn('w:fldCharType'), 'end')

run._r.append(fldChar1)
run._r.append(instrText)
run._r.append(fldChar2)
run._r.append(fldChar3)
run._r.append(fldChar4)

# Center the footer paragraph
paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

In [15]:
doc.save('english_stories.docx')

In [16]:
from googletrans import Translator

# Create a Translator object
translator = Translator()

doc2 = Document()

with open('stories.json', 'r') as f:
    english_stories = json.load(f)

for i in range(len(english_stories)):
  heading3 = doc2.add_heading(f"{i + 1}", level=1)
  # Center the heading
  heading3.alignment = WD_ALIGN_PARAGRAPH.CENTER
  # Increase the font size
  heading3.style.font.size = Pt(24)

  heading4 = doc2.add_heading(f"{ (translator.translate(english_stories[i]['story_place'], src='en', dest='ar').text)}", level=1)
  # Center the heading
  heading4.alignment = WD_ALIGN_PARAGRAPH.CENTER
  # Increase the font size
  heading4.style.font.size = Pt(24)

  # Insert a line break directly after the heading
  run = heading4.add_run()
  run.add_break()

  stories_num = english_stories[i]['number']
  picture2 = doc2.add_picture(f'/content/images/{stories_num}.jpg', width=Inches(3))
  # Center the picture.
  picture2.alignment = WD_ALIGN_PARAGRAPH.CENTER

  # Center the paragraph that contains the picture
  last_paragraph2 = doc2.paragraphs[-1]  # Get the last paragraph (where the picture was added)
  last_paragraph2.alignment = WD_ALIGN_PARAGRAPH.CENTER

  paragraph2 = doc2.add_paragraph(english_stories[i]['translation'])
  # paragraph2.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY  # Justify alignment

  # Setting paragraph from right to left.
  paragraph2.alignment = 2


  # Set the line spacing to 1.5
  paragraph_format = paragraph2.paragraph_format
  paragraph_format.line_spacing = Pt(18)  # 1.5 lines

  # # Center the paragraph horizontally
  paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER

  # # Modify the section properties for vertical alignment
  # section = doc.sections[-1]  # Get the last section
  # section.start_type = WD_SECTION_START.CONTINUOUS
  # section.orientation = WD_ORIENTATION.PORTRAIT
  # section.page_height = section.page_height
  # section.page_width = section.page_width

  # # Set vertical alignment to center
  # section.vertical_alignment = WD_ALIGN_PARAGRAPH.CENTER

  # Set space before and after the paragraph
  paragraph_format.space_before = Pt(12)  # Space before the paragraph
  paragraph_format.space_after = Pt(12)   # Space after the paragraph

  # Set the font size and font type
  for run in paragraph2.runs:
      run.font.size = Pt(12)           # Set the font size to 12 points
      run.font.name = 'Cascadia Mono'          # Set the font type to Arial
  # if i != len(english_stories) - 1:
  #   doc.add_page_break()

In [ ]:
# Add a footer with page numbering
section = doc2.sections[0]
footer = section.footer
paragraph = footer.paragraphs[0]

# Create the field code for page numbering
field_code = 'PAGE  \\* MERGEFORMAT'
run = paragraph.add_run()
fldChar1 = OxmlElement('w:fldChar')
fldChar1.set(qn('w:fldCharType'), 'begin')

instrText = OxmlElement('w:instrText')
instrText.set(qn('xml:space'), 'preserve')
instrText.text = field_code

fldChar2 = OxmlElement('w:fldChar')
fldChar2.set(qn('w:fldCharType'), 'separate')

fldChar3 = OxmlElement('w:t')
fldChar3.text = "1"

fldChar4 = OxmlElement('w:fldChar')
fldChar4.set(qn('w:fldCharType'), 'end')

run._r.append(fldChar1)
run._r.append(instrText)
run._r.append(fldChar2)
run._r.append(fldChar3)
run._r.append(fldChar4)

# Center the footer paragraph
paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

In [ ]:
doc2.save('arabic_stories.docx')